In [11]:
from pandas import read_csv
from pandas import to_datetime
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARIMA
from matplotlib import pyplot

def parser(x):
    return to_datetime('199'+x,'%Y-%m')
series = read_csv('./dl_pytorch/chap07/data/SBUX.csv',header=0,parse_dates=[0],index_col=0,squeeze=True,
                  date_parser=parser)
model=ARIMA(series,order=(5,1,0))
model_fit = model.fit(disp=0)
print(model_fit.summary())
residuals = DataFrame(model_fit.resid)
residuals.plot()
pyplot.show()
residuals.plot(kind='kde')
pyplot.show()
print(residuals.describe)

C:\Users\user\AppData\Local\Temp\ipykernel_14308\2600277882.py:9: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  series = read_csv('./dl_pytorch/chap07/data/SBUX.csv',header=0,parse_dates=[0],index_col=0,squeeze=True,
C:\Users\user\AppData\Local\Temp\ipykernel_14308\2600277882.py:9: FutureWarning: 
        Use pd.to_datetime instead.

  series = read_csv('./dl_pytorch/chap07/data/SBUX.csv',header=0,parse_dates=[0],index_col=0,squeeze=True,


AssertionError: 

7.4.1 RNN 셀 구현

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchtext
import time

In [2]:
start = time.time()
TEXT = torchtext.legacy.data.Field(lower=True,fix_length=200,batch_first=False)
LABEL = torchtext.legacy.data.Field(sequential=False)

In [3]:
from torchtext.legacy import datasets
train_data,test_data =datasets.IMDB.splits(TEXT,LABEL)

In [8]:
print(vars(train_data.examples[0]))

{'text': ['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy.', 'it', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life,', 'such', 'as', '"teachers".', 'my', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'bromwell', "high's", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"teachers".', 'the', 'scramble', 'to', 'survive', 'financially,', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', "teachers'", 'pomp,', 'the', 'pettiness', 'of', 'the', 'whole', 'situation,', 'all', 'remind', 'me', 'of', 'the', 'schools', 'i', 'knew', 'and', 'their', 'students.', 'when', 'i', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school,', 'i', 'immediately', 'recalled', '.........', 'at', '..........', 'high.', 'a', 'classic', 'line:', 'inspector:', "i'm", 'here', 'to', 'sack', 'one', 'of', '

In [11]:
import string
for example in train_data.examples:
    text = [x.lower() for x in vars(example)['text']] # 소문자
    text = [x.replace("<br","") for x in text] # <br -> 공백
    text = [''.join(c for c in s if c not in string.punctuation) for s in text] # 구두점 제거
    text = [s for s in text if s] #공백 제거
    vars(example)['text']=text

In [13]:
import random
train_data, valid_data = train_data.split(random_state=random.seed(0),split_ratio=0.8)

In [14]:
print(f'Number of training examples : {len(train_data)}')
print(f'Number of validation examples : {len(valid_data)}')
print(f'Number of testing examples:{len(test_data)}')


Number of training examples : 20000
Number of validation examples : 5000
Number of testing examples:25000


In [15]:
TEXT.build_vocab(train_data,max_size=10000,min_freq=10,vectors=None)
LABEL.build_vocab(train_data)

print(f'Unique tokens in TEXT vocabulary:{len(TEXT.vocab)}')
print(f'Unique tokens in LABEL vocabulary:{len(LABEL.vocab)}')

Unique tokens in TEXT vocabulary:10002
Unique tokens in LABEL vocabulary:3


In [16]:
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x000002A4CDBDFDF0>>, {'<unk>': 0, 'pos': 1, 'neg': 2})


In [17]:
BATCH_SIZE=32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_dim = 100
hidden_size = 300

train_iterator, valid_iterator,test_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data,valid_data,test_data),
    batch_size=BATCH_SIZE,
    device=device
)

In [19]:
class RNNCell_Encoder(nn.Module):
    def __init__(self,input_dim,hidden_size):
        super(RNNCell_Encoder,self).__init__()
        self.rnn = nn.RNNCell(input_dim,hidden_size)

    def forward(self,inputs):
        bz = inputs.shape[1]
        ht = torch.zeros((bz,hidden_size)).to(device)
        for word in inputs:
            ht = self.rnn(word,ht) # 재귀 처리
        return ht

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.em = nn.Embedding(len(TEXT.vocab.stoi),embedding_dim)
        self.rnn = RNNCell_Encoder(embedding_dim,hidden_size)
        self.fc1 = nn.Linear(hidden_size,256)
        self.fc2 = nn.Linear(256,3)

    def forward(self,x):
        x = self.em(x)
        x = self.rnn(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [20]:
model = Net()
model.to(device)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)

In [21]:
def training(epoch,model,trainloader,validloader):
    correct = 0
    total = 0
    running_loss =0

    model.train()
    for b in trainloader:
        x,y = b.text,b.label
        x,y = x.to(device),y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred,dim=1)
            correct +=(y_pred==y).sum().item()
            total +=y.size(0)
            running_loss +=loss.item()

    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct/total

    valid_correct = 0
    valid_total = 0
    valid_running_loss = 0

    model.eval()
    with torch.no_grad():
        for b in validloader:
            x,y = b.text,b.label
            x,y = x.to(device),y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred,y)
            y_pred = torch.argmax(y_pred,dim=1)
            valid_correct +=(y_pred==y).sum().item()
            valid_total +=y.size(0)
            valid_running_loss +=loss.item()
    epoch_valid_loss = valid_running_loss / len(validloader.dataset)
    epoch_valid_acc = valid_correct/valid_total

    print('epoch: ',epoch,
          'loss: ',round(epoch_loss,3),
          'accuracy',round(epoch_acc,3),
          'valid_loss: ',round(epoch_valid_loss,3),
          'valid_acc : ',round(epoch_valid_acc,3))
    return epoch_loss,epoch_acc,epoch_valid_loss,epoch_valid_acc

In [22]:
epochs = 5
train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(epochs):
    epoch_loss,epoch_acc,epoch_valid_loss,epoch_valid_acc = training(epoch,model,train_iterator,valid_iterator)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    valid_loss.append(epoch_valid_loss)
    valid_acc.append(epoch_valid_acc)
end = time.time()
print(end-start)

epoch:  0 loss:  0.022 accuracy 0.497 valid_loss:  0.022 valid_acc :  0.491
epoch:  1 loss:  0.022 accuracy 0.503 valid_loss:  0.022 valid_acc :  0.515
epoch:  2 loss:  0.022 accuracy 0.513 valid_loss:  0.022 valid_acc :  0.517
epoch:  3 loss:  0.022 accuracy 0.529 valid_loss:  0.022 valid_acc :  0.517
epoch:  4 loss:  0.021 accuracy 0.532 valid_loss:  0.022 valid_acc :  0.5
2714.220121383667


In [23]:
print(torch.cuda.is_available())

False
